In [1]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from urllib.request import urlopen
import ssl
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
import time
from pathlib import Path
import os

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&pa=0&si=&cn='
driver.get(url)
page = driver.page_source
bs_obj = BS(page, 'html.parser')
table = bs_obj.select('.nav-tabs-custom')[0]
head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
element3 = []
while len(head1) == 81:
    page = driver.page_source
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2.remove('WAR*')
    head2.remove('WPA')
    head2 += ['WAR*', 'WPA']
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/35*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    try:
        driver.find_element(By.LINK_TEXT, '다음') != None
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        time.sleep(1)
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        next_link.click()
        head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
    except:
        print('수집 종료')
        break
else:
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END)
    next_link = driver.find_element(By.LINK_TEXT, '마지막')
    next_link.click()
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2.remove('WAR*')
    head2.remove('WPA')
    head2 += ['WAR*', 'WPA']
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/35*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    print('수집 종료')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\admin\.wdm\drivers\chromedriver\win32\103.0.5060.53]


수집 종료


In [3]:
df1 = pd.DataFrame(element3, columns=head2)
df1

,순,이름,팀,정렬,G,타석,타수,득점,안타,2타,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,이정후,22키CF,5.11,85,369,320,44,106,19,...,0,2,.331,.415,.556,.971,.427,175.5,5.11,3.92
1,2,피렐라,22삼LF,4.36,82,360,318,59,108,17,...,0,0,.340,.417,.572,.989,.439,173.7,4.36,1.81
2,3,나성범,22KRF,4.04,83,375,315,51,97,25,...,0,4,.308,.408,.514,.922,.415,163.1,4.04,2.47
3,4,소크라테스,22KCF,3.97,76,328,304,54,101,20,...,0,1,.332,.378,.546,.924,.412,161.8,3.97,1.18
4,5,김현수,22LLF,3.55,83,361,321,55,93,15,...,0,0,.290,.368,.520,.889,.395,158.2,3.55,2.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,307,오영수,22N1B,-0.69,44,136,124,9,24,6,...,0,2,.194,.250,.290,.540,.253,51.8,-0.69,-0.50
307,308,정보근,22롯C,-0.74,54,137,123,5,21,2,...,6,0,.171,.221,.187,.408,.192,8.3,-0.74,-1.19
308,309,박경수,22K2B,-0.78,59,103,90,6,10,1,...,0,0,.111,.223,.122,.346,.185,5.5,-0.78,-1.24
309,310,조세진,22롯RF,-0.88,32,81,80,5,13,3,...,1,0,.163,.163,.200,.363,.165,-10.4,-0.88,-0.64


In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=WRCPLUS&o2=WAR_ALL&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
driver.get(url)
page = driver.page_source
bs_obj = BS(page, 'html.parser')
table = bs_obj.select('.nav-tabs-custom')[0]
head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
element3 = []
while len(head1) == 57:
    page = driver.page_source
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/27*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    try:
        driver.find_element(By.LINK_TEXT, '다음') != None
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        time.sleep(1)
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        next_link.click()
        head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
    except:
        print('수집 종료')
        break
else:
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END)
    next_link = driver.find_element(By.LINK_TEXT, '마지막')
    next_link.click()
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/27*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    print('수집 종료')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


수집 종료


In [5]:
df2 = pd.DataFrame(element3, columns=head2)
df2 = (df2
       .iloc[:, 0:14]
       .assign(**{'wRC+':df2.iloc[:, -1]})
      )
df2

,순,이름,팀,정렬,타석,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN,wRC+
0,1,문상준,22KSS,327.4,2,0.00,0.0,50.0,0.00,.500,.000,1.000,1.9,,327.4
1,2,김민혁,22두,216.0,10,10.00,0.0,20.0,0.00,.333,.067,.333,3.9,0.00,216.0
2,3,강현구,22두LF,208.9,5,0.00,20.0,0.0,,.000,.267,.333,1.9,,208.9
3,4,전의산,22S1B,201.6,103,6.80,11.7,24.3,0.48,.341,.077,.407,3.6,0.00,201.6
4,5,권혁경,22KC,199.5,2,0.00,0.0,0.0,,.000,.000,.500,1.9,,199.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,,김재유,22롯,,0,,,,,,,,,,
307,,권민석,22두,,0,,,,,,,,,,
308,,박정우,22K,,0,,,,,,,,,,
309,,전민재,22두DH,,2,0.00,0.0,100.0,0.00,.000,.000,,,,


In [6]:
def numeric(df):
    for col in df.columns.to_list():
        df[col] = pd.to_numeric(df[col], errors='ignore')
    return df

In [7]:
df_total = (df1.merge(df2, on=['이름', '팀', '타석', 'wRC+'], how='inner')
)
df_total = (df_total
 .drop(columns=[col for col in df_total.columns if '_x' in col or '_y' in col])
            .drop_duplicates(subset=['이름', '팀', '타석', 'wRC+'])
            .reset_index(drop=True)
            .fillna(0.00)
            .replace('', 0.00)
            .pipe(numeric)
)
df_total

,이름,팀,G,타석,타수,득점,안타,2타,3타,홈런,...,WPA,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,이정후,22키CF,85,369,320,44,106,19,4,15,...,3.92,4.07,11.9,4.9,2.44,0.225,0.083,0.315,4.2,3.53
1,피렐라,22삼LF,82,360,318,59,108,17,3,17,...,1.81,4.72,9.2,10.8,0.85,0.233,0.077,0.347,4.6,9.92
2,나성범,22KRF,83,375,315,51,97,25,2,12,...,2.47,3.20,11.5,21.1,0.54,0.206,0.100,0.373,4.6,6.00
3,소크라테스,22KCF,76,328,304,54,101,20,6,11,...,1.18,3.35,6.4,15.9,0.40,0.214,0.046,0.372,6.3,9.90
4,김현수,22LLF,83,361,321,55,93,15,1,19,...,2.17,5.26,10.0,10.0,1.00,0.231,0.079,0.278,3.7,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,오영수,22N1B,44,136,124,9,24,6,0,2,...,-0.50,1.47,6.6,26.5,0.25,0.097,0.056,0.250,3.0,0.00
287,정보근,22롯C,54,137,123,5,21,2,0,0,...,-1.19,0.00,5.8,21.9,0.27,0.016,0.051,0.226,2.2,0.00
288,박경수,22K2B,59,103,90,6,10,1,0,0,...,-1.24,0.00,12.6,30.1,0.42,0.011,0.112,0.170,1.3,0.00
289,조세진,22롯RF,32,81,80,5,13,3,0,0,...,-0.64,0.00,0.0,29.6,0.00,0.038,0.000,0.232,2.2,0.00


In [8]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
driver.get(url)
page = driver.page_source
bs_obj = BS(page, 'html.parser')
table = bs_obj.select('.nav-tabs-custom')[0]
head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
element3 = []
while len(head1) == 87:
    page = driver.page_source
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2.remove('WAR')
    head2.remove('WPA')
    head2 += ['WAR', 'WPA']
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/37*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    try:
        driver.find_element(By.LINK_TEXT, '다음') != None
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        time.sleep(1)
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        next_link.click()
        head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
    except:
        print('수집 종료')
        break
else:
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END)
    next_link = driver.find_element(By.LINK_TEXT, '마지막')
    next_link.click()
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    head2.remove('WAR')
    head2.remove('WPA')
    head2 += ['WAR', 'WPA']
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/37*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    print('수집 종료')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


수집 종료


In [9]:
df3 = pd.DataFrame(element3, columns=head2)
df3

,순,이름,팀,정렬,출장,완투,완봉,선발,승,패,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,1,폰트,22S,5.24,18,1,0,18,11,4,...,114,0,4,1.96,2.90,0.77,209.5,143.4,5.24,3.41
1,2,안우진,22키,4.17,17,0,0,17,10,4,...,125,0,5,2.02,2.29,0.96,195.4,171.5,4.17,3.13
2,3,김광현,22S,3.70,15,0,0,15,9,1,...,85,0,1,1.65,2.74,1.05,248.6,150.7,3.70,2.17
3,4,요키시,22키,3.69,17,0,0,17,7,4,...,94,0,2,2.48,2.83,1.00,159.3,137.9,3.69,2.30
4,5,고영표,22K,3.61,16,1,1,16,7,5,...,101,0,0,2.90,2.43,1.09,139.8,164.7,3.61,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,240,노바,22S,-0.81,12,0,0,12,3,4,...,32,1,9,6.50,4.77,1.62,63.1,86.3,-0.81,-0.80
240,241,김윤수,22삼,-0.85,28,0,0,0,2,2,...,21,0,6,6.46,4.01,1.69,63.7,100.7,-0.85,-0.14
241,242,주현상,22한,-0.87,33,0,0,0,0,1,...,25,0,0,7.98,4.48,1.75,49.7,89.7,-0.87,-1.05
242,243,황동재,22삼,-0.96,12,0,0,9,1,2,...,37,1,0,6.90,5.09,1.86,59.7,81.4,-0.96,-1.29


In [10]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP&o2=WAR&de=0&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
driver.get(url)
page = driver.page_source
bs_obj = BS(page, 'html.parser')
table = bs_obj.select('.nav-tabs-custom')[0]
head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
element3 = []
while len(head1) == 78:
    page = driver.page_source
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    order_change = ['WHIP', 'WHIP+', '투구', 'IP/G', 'P/G', 'P/IP', 'P/PA', 'CYP']
    for change in order_change:
        head2.remove(change)
    head2 += order_change
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/34*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    try:
        driver.find_element(By.LINK_TEXT, '다음') != None
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        time.sleep(1)
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        next_link.click()
        head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
    except:
        print('수집 종료')
        break
else:
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END)
    next_link = driver.find_element(By.LINK_TEXT, '마지막')
    next_link.click()
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = head1[0:int(len(head1)/3)]
    order_change = ['WHIP', 'WHIP+', '투구', 'IP/G', 'P/G', 'P/IP', 'P/PA', 'CYP']
    for change in order_change:
        head2.remove(change)
    head2 += order_change
    head2 = ['순', '이름', '팀', '정렬'] + head2
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/34*4):-2]
    for i in range(0, len(element2), len(head2)):
        element3.append(element2[i:i+len(head2)])
    print('수집 종료')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


수집 종료


In [11]:
df4 = pd.DataFrame(element3, columns=head2)
df4

,순,이름,팀,정렬,출장,이닝,ERA,FIP,K/9,BB/9,...,장타율,OPS,WHIP,WHIP+,투구,IP/G,P/G,P/IP,P/PA,CYP
0,1,문승원,22S,0.34,1,1.1,0.00,0.34,13.50,0.00,...,.000,.000,0.00,0.00,9,1.3,9.0,6.8,2.25,0.9
1,2,김태오,22K,1.15,4,3.2,4.91,1.15,9.82,0.00,...,.389,.778,1.91,1.91,60,0.9,15.0,16.4,3.33,0.2
2,3,남하준,22K,1.34,1,1.0,0.00,1.34,9.00,0.00,...,.400,.800,2.00,2.00,14,1.0,14.0,14.0,2.80,0.6
3,4,김재윤,22K,2.05,33,35.2,2.78,2.05,11.36,2.52,...,.234,.466,0.90,0.90,514,1.1,15.6,14.4,3.67,14.3
4,5,이우찬,22L,2.06,12,15.2,1.15,2.06,12.06,1.72,...,.246,.459,0.77,0.83,238,1.3,19.8,15.2,3.90,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,240,이강준,22롯,11.67,6,3.0,15.00,11.67,3.00,27.00,...,.308,.899,4.33,4.33,95,0.5,15.8,31.7,4.32,-3.4
240,241,이재민,22한,12.34,2,2.0,36.00,12.34,0.00,22.50,...,.500,1.100,4.00,4.50,55,1.0,27.5,27.5,3.67,-7.0
241,242,허준혁,22L,13.43,5,3.2,12.27,13.43,9.82,4.91,...,1.050,1.550,3.00,3.00,79,0.7,15.8,21.6,3.59,-2.8
242,243,김태훈,22S,13.46,9,5.1,10.13,13.46,1.69,16.88,...,.684,1.218,3.00,3.00,118,0.6,13.1,22.1,3.93,-3.2


In [12]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = 'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2022&ye=2022&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=14&o1=FVval&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
driver.get(url)
page = driver.page_source
bs_obj = BS(page, 'html.parser')
table = bs_obj.select('.nav-tabs-custom')[0]
head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
element3 = []
while len(head1) == 75:
    page = driver.page_source
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = ['순', '이름', '팀', '직구평속', '출장', '이닝', '직구평속', '슬라평속', '커브평속', '첸졉평속', '스플평속', '투심평속', '너클평속', '기타평속']
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/33*4):-2]
    for i in range(0, len(element2), 29):
        element3.append(element2[i:i+29])
        element3 = [element[0:len(head2)] for element in element3]
    try:
        driver.find_element(By.LINK_TEXT, '다음') != None
        html = driver.find_element(By.TAG_NAME, 'html')
        html.send_keys(Keys.END)
        time.sleep(1)
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        next_link.click()
        head1 = [element.text for element in table.find_all('a', {'style':'color:black;'})]
    except:
        print('수집 종료')
        break
else:
    html = driver.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.END)
    next_link = driver.find_element(By.LINK_TEXT, '마지막')
    next_link.click()
    bs_obj = BS(page, 'html.parser')
    table = bs_obj.select('.nav-tabs-custom')[0]
    head2 = ['순', '이름', '팀', '직구평속', '출장', '이닝', '직구평속', '슬라평속', '커브평속', '첸졉평속', '스플평속', '투심평속', '너클평속', '기타평속']
    element1 = [element.text.strip() for element in table.find_all('td')]
    element2 = element1[int((len(element1)-2)/33*4):-2]
    for i in range(0, len(element2), 29):
        element3.append(element2[i:i+29])
        element3 = [element[0:len(head2)] for element in element3]
    print('수집 종료')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


수집 종료


In [13]:
df5 = pd.DataFrame(element3, columns=head2)
max_speed = (df5
 .replace('', 0.00)
 .iloc[:, 6:]
 .astype('float64')
 .max(axis=1)
)
df5 = (df5
       .assign(평속=max_speed)
       .loc[:, ['이름', '팀', '출장', '이닝', '평속']]
      )
df5

,이름,팀,출장,이닝,평속
0,안우진,22키,17,111.1,153.5
1,조요한,22S,18,16.1,153.3
2,고우석,22L,38,36.0,152.9
3,스탁,22두,18,103.2,153.0
4,문동주,22한,10,13.2,152.2
...,...,...,...,...,...
239,박준표,22K,9,6.1,139.8
240,하준수,22N,1,1.0,144.5
241,전유수,22K,5,8.2,137.2
242,페냐,22한,3,12.2,148.4


In [14]:
df_total2 = (df3
             .merge(df4, on=['이름', '팀', '출장', '이닝', 'ERA', 'FIP', 'WHIP'], how='inner')
             .merge(df5, on=['이름', '팀', '출장', '이닝'], how='inner')
            )
df_total2 = (df_total2
             .drop(columns=[col for col in df_total2.columns if '_x' in col or '_y' in col])
             .drop_duplicates(subset=['이름', '팀', '출장', '이닝', 'ERA', 'FIP'])
             .reset_index(drop=True)
             .fillna(0.00)
             .replace('', 0.00)
             .pipe(numeric)
)
df_total2

,이름,팀,출장,완투,완봉,선발,승,패,세,홀드,...,장타율,OPS,WHIP+,투구,IP/G,P/G,P/IP,P/PA,CYP,평속
0,폰트,22S,18,1,0,18,11,4,0,0,...,0.280,0.489,0.78,1761,6.9,97.8,14.2,3.80,57.4,148.6
1,안우진,22키,17,0,0,17,10,4,0,0,...,0.263,0.514,0.99,1735,6.6,102.1,15.6,3.95,53.2,153.5
2,김광현,22S,15,0,0,15,9,1,0,0,...,0.292,0.561,1.07,1385,6.2,92.3,15.0,3.72,46.8,145.2
3,요키시,22키,17,0,0,17,7,4,0,0,...,0.335,0.599,1.03,1615,6.2,95.0,15.3,3.88,40.1,142.8
4,고영표,22K,16,1,1,16,7,5,0,0,...,0.317,0.612,1.21,1557,6.8,97.3,14.3,3.51,36.4,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,노바,22S,12,0,0,12,3,4,0,0,...,0.416,0.790,1.70,1046,5.3,87.2,16.4,3.60,-8.0,147.7
240,김윤수,22삼,28,0,0,0,2,2,0,3,...,0.379,0.747,1.78,439,0.9,15.7,18.6,3.85,-1.1,151.1
241,주현상,22한,33,0,0,0,0,1,0,2,...,0.544,0.942,1.83,679,1.2,20.6,17.7,3.82,-12.3,143.4
242,황동재,22삼,12,0,0,9,1,2,0,0,...,0.510,0.904,1.86,929,3.8,77.4,20.3,4.28,-7.5,138.8


In [15]:
df1_columns = df_total.columns.to_list()
df2_columns = df_total2.columns.to_list()
df1_columns
df2_columns

['이름',
 '팀',
 '출장',
 '완투',
 '완봉',
 '선발',
 '승',
 '패',
 '세',
 '홀드',
 '이닝',
 '실점',
 '자책',
 '타자',
 '안타',
 '2타',
 '3타',
 '홈런',
 '볼넷',
 '고4',
 '사구',
 '삼진',
 '보크',
 '폭투',
 'ERA',
 'FIP',
 'WHIP',
 'ERA+',
 'FIP+',
 'WAR',
 'WPA',
 'K/9',
 'BB/9',
 'K/BB',
 'HR/9',
 'K%',
 'BB%',
 'K-BB%',
 'PFR',
 'BABIP',
 'LOB%',
 '타율',
 '출루율',
 '장타율',
 'OPS',
 'WHIP+',
 '투구',
 'IP/G',
 'P/G',
 'P/IP',
 'P/PA',
 'CYP',
 '평속']

In [16]:
df1_columns = ['연도', '이름', '소속', '포지션', 'G', '타석', '타수', '득점', '안타', '2타', '3타', 
               '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', 
               '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA', 'HR%', 'BB%', 'K%', 
               'BB/K', 'IsoP', 'IsoD', 'BABIP', 'Spd', 'PSN']

In [17]:
df1_total = (df_total
 .assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
         포지션=lambda df: df['팀'].str.extract('(LF|CF|RF|1B|2B|3B|SS|C|DH)'),
         소속=lambda df: df['팀'].str.slice(2, 4).str.extract('(L|키|K|S|N|한|삼|롯|두|넥)'))
            .loc[:, df1_columns]
)
df_total

,이름,팀,G,타석,타수,득점,안타,2타,3타,홈런,...,WPA,HR%,BB%,K%,BB/K,IsoP,IsoD,BABIP,Spd,PSN
0,이정후,22키CF,85,369,320,44,106,19,4,15,...,3.92,4.07,11.9,4.9,2.44,0.225,0.083,0.315,4.2,3.53
1,피렐라,22삼LF,82,360,318,59,108,17,3,17,...,1.81,4.72,9.2,10.8,0.85,0.233,0.077,0.347,4.6,9.92
2,나성범,22KRF,83,375,315,51,97,25,2,12,...,2.47,3.20,11.5,21.1,0.54,0.206,0.100,0.373,4.6,6.00
3,소크라테스,22KCF,76,328,304,54,101,20,6,11,...,1.18,3.35,6.4,15.9,0.40,0.214,0.046,0.372,6.3,9.90
4,김현수,22LLF,83,361,321,55,93,15,1,19,...,2.17,5.26,10.0,10.0,1.00,0.231,0.079,0.278,3.7,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,오영수,22N1B,44,136,124,9,24,6,0,2,...,-0.50,1.47,6.6,26.5,0.25,0.097,0.056,0.250,3.0,0.00
287,정보근,22롯C,54,137,123,5,21,2,0,0,...,-1.19,0.00,5.8,21.9,0.27,0.016,0.051,0.226,2.2,0.00
288,박경수,22K2B,59,103,90,6,10,1,0,0,...,-1.24,0.00,12.6,30.1,0.42,0.011,0.112,0.170,1.3,0.00
289,조세진,22롯RF,32,81,80,5,13,3,0,0,...,-0.64,0.00,0.0,29.6,0.00,0.038,0.000,0.232,2.2,0.00


In [18]:
df2_columns = ['연도', '이름', '소속', '출장', '완투', '완봉', '선발', '승', '패', '세', '홀드',
 '이닝', '실점', '자책', '타자', '안타', '2타', '3타', '홈런', '볼넷', '고4', '사구', '삼진', '보크',
 '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA', 'K/9', 'BB/9', 'K/BB', 'HR/9', 'K%', 'BB%', 'K-BB%', 'PFR',
 'BABIP', 'LOB%', '타율', '출루율', '장타율', 'OPS', 'WHIP+', '투구', 'IP/G', 'P/G', 'P/IP', 'P/PA', 'CYP', '평속']

In [19]:
df2_total = (df_total2
 .assign(연도=lambda df: '20' + df['팀'].str.slice(0, 2),
        소속=lambda df: df['팀'].str.extract('(L|키|K|S|N|한|삼|롯|두|넥)'))
 .loc[:, df2_columns]
)
df_total2

,이름,팀,출장,완투,완봉,선발,승,패,세,홀드,...,장타율,OPS,WHIP+,투구,IP/G,P/G,P/IP,P/PA,CYP,평속
0,폰트,22S,18,1,0,18,11,4,0,0,...,0.280,0.489,0.78,1761,6.9,97.8,14.2,3.80,57.4,148.6
1,안우진,22키,17,0,0,17,10,4,0,0,...,0.263,0.514,0.99,1735,6.6,102.1,15.6,3.95,53.2,153.5
2,김광현,22S,15,0,0,15,9,1,0,0,...,0.292,0.561,1.07,1385,6.2,92.3,15.0,3.72,46.8,145.2
3,요키시,22키,17,0,0,17,7,4,0,0,...,0.335,0.599,1.03,1615,6.2,95.0,15.3,3.88,40.1,142.8
4,고영표,22K,16,1,1,16,7,5,0,0,...,0.317,0.612,1.21,1557,6.8,97.3,14.3,3.51,36.4,137.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,노바,22S,12,0,0,12,3,4,0,0,...,0.416,0.790,1.70,1046,5.3,87.2,16.4,3.60,-8.0,147.7
240,김윤수,22삼,28,0,0,0,2,2,0,3,...,0.379,0.747,1.78,439,0.9,15.7,18.6,3.85,-1.1,151.1
241,주현상,22한,33,0,0,0,0,1,0,2,...,0.544,0.942,1.83,679,1.2,20.6,17.7,3.82,-12.3,143.4
242,황동재,22삼,12,0,0,9,1,2,0,0,...,0.510,0.904,1.86,929,3.8,77.4,20.3,4.28,-7.5,138.8


In [20]:
p = Path.cwd() / 'new_folder'
writer = pd.ExcelWriter(p / 'Statiz_2022.xlsx', engine='xlsxwriter')
df2_total.to_excel(writer, sheet_name='투수', index=False)
df1_total.to_excel(writer, sheet_name='타자', index=False)
writer.save()